In [2]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")


# Subset data for the summer months and visualize

In [41]:

df = pd.read_csv('hourly.csv')
df['Date'] = pd.to_datetime(df['DATE'])
df['Month'] = df['Date'].dt.month
summer_data = df[(df['Month'] >= 6) & (df['Month'] <= 8)]
subsetted_data = summer_data.iloc[:, [1, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16]]
print(subsetted_data.head(10))
print(subsetted_data.columns)

                     DATE  LATITUDE  LONGITUDE  ELEVATION             WND  \
4918  2022-06-01T00:00:00  29.98438  -95.36072       27.5  140,1,N,0088,1   
4919  2022-06-01T00:53:00  29.98438  -95.36072       27.5  130,1,N,0082,1   
4920  2022-06-01T01:53:00  29.98438  -95.36072       27.5  120,1,N,0062,1   
4921  2022-06-01T02:53:00  29.98438  -95.36072       27.5  130,1,N,0051,1   
4922  2022-06-01T03:53:00  29.98438  -95.36072       27.5  140,1,N,0041,1   
4923  2022-06-01T04:53:00  29.98438  -95.36072       27.5  160,1,N,0031,1   
4924  2022-06-01T05:20:00  29.98438  -95.36072       27.5  150,1,N,0031,1   
4925  2022-06-01T05:53:00  29.98438  -95.36072       27.5  150,1,N,0036,1   
4926  2022-06-01T05:59:00  29.98438  -95.36072       27.5  999,9,9,9999,9   
4927  2022-06-01T05:59:00  29.98438  -95.36072       27.5  999,9,9,9999,9   

              CIG           VIS      TMP      DEW      SLP          AA1  
4918  22000,1,9,N  016000,1,9,9  +0300,1  +0222,1  10117,1          NaN  
4919

# Format the data based on the conditions and create additional columns a necessary

In [42]:
# Split the 'WND' column into 'DIR', 'DIR_QC', 'TYPE', 'SPD', 'SPD_QC' columns
subsetted_data[['DIR', 'DIR_QC', 'WIND_TYPE', 'SPD', 'SPD_QC']] = subsetted_data['WND'].str.split(',', expand=True)

# Split the 'CIG' column into 'HEIGHT_AGL', 'HEIGHT_QC', 'UNNECESSARY_1', 'UNNECESSARY_2'
subsetted_data[['HEIGHT_AGL', 'HEIGHT_QC', 'UNNECESSARY_1', 'UNNECESSARY_2']] = subsetted_data['CIG'].str.split(',', expand=True)

# Split the 'VIS' column into 'VIS', 'VIS_QC', 'Unused1', 'Unused2' columns
subsetted_data[['VIS', 'VIS_QC', 'Unused1', 'Unused2']] = subsetted_data['VIS'].str.split(',', expand=True)

subsetted_data[['TMP', 'TMP_QC']] = subsetted_data['TMP'].str.split(',', expand=True)
subsetted_data[['DEW', 'DEW_QC']] = subsetted_data['DEW'].str.split(',', expand=True)
subsetted_data[['SLP', 'SLP_QC']] = subsetted_data['SLP'].str.split(',', expand=True)
subsetted_data[['PRECIP_HOURS', 'PRECIP_AMOUNT', 'Col3', 'Col4']] = subsetted_data['AA1'].str.split(',', expand=True)

# Remove the 'WND' column
subsetted_data.drop('WND', axis=1, inplace=True)
subsetted_data.drop('CIG', axis=1, inplace=True)

# Process direction based on direction quality code
subsetted_data.loc[subsetted_data['DIR_QC'].isin(['2', '3', '6', '7']), 'DIR'] = '999'
subsetted_data.drop('DIR_QC', axis=1, inplace=True)

# Process speed based on speed quality code
subsetted_data.loc[subsetted_data['SPD_QC'].isin(['2', '3', '6', '7']), 'SPD'] = '999'
subsetted_data.drop('SPD_QC', axis=1, inplace=True)
subsetted_data.drop('WIND_TYPE', axis=1, inplace=True)
subsetted_data.drop('UNNECESSARY_1', axis=1, inplace=True)
subsetted_data.drop('UNNECESSARY_2', axis=1, inplace=True)

# Process height AGL based on height quality code
subsetted_data.loc[subsetted_data['HEIGHT_QC'].isin(['2', '3', '6', '7']), 'HEIGHT_AGL'] = '99999'
subsetted_data.drop('HEIGHT_QC', axis=1, inplace=True)

# Process visibility based on visibility quality code
subsetted_data.loc[subsetted_data['VIS_QC'].isin(['2', '3', '6', '7']), 'VIS'] = '999999'
subsetted_data.drop('VIS_QC', axis=1, inplace=True)
subsetted_data.drop('Unused1', axis=1, inplace=True)
subsetted_data.drop('Unused2', axis=1, inplace=True)

subsetted_data.loc[subsetted_data['TMP_QC'].isin(['2', '3', '6', '7']), 'TMP'] = '9999'
subsetted_data.drop('TMP_QC', axis=1, inplace=True)

subsetted_data.loc[subsetted_data['DEW_QC'].isin(['2', '3', '6', '7']), 'DEW'] = '9999'
subsetted_data.drop('DEW_QC', axis=1, inplace=True)

subsetted_data.loc[subsetted_data['SLP_QC'].isin(['2', '3', '6', '7']), 'SLP'] = '99999'
subsetted_data.drop('SLP_QC', axis=1, inplace=True)

subsetted_data.loc[subsetted_data['Col4'].isin(['2', '3', '6', '7']), 'PRECIP_AMOUNT'] = '0'
subsetted_data.drop(['Col3', 'Col4', 'AA1'], axis=1, inplace=True)
subsetted_data.rename(columns={'PRECIP_AMOUNT': 'PRECIP'}, inplace=True)

# Replace NaN values with 9999
subsetted_data['PRECIP'].fillna(0, inplace=True)
subsetted_data['PRECIP_HOURS'].fillna(00, inplace=True)

# Display the modified DataFrame
print(subsetted_data.head(10))

                     DATE  LATITUDE  LONGITUDE  ELEVATION     VIS    TMP  \
4918  2022-06-01T00:00:00  29.98438  -95.36072       27.5  016000  +0300   
4919  2022-06-01T00:53:00  29.98438  -95.36072       27.5  016093  +0283   
4920  2022-06-01T01:53:00  29.98438  -95.36072       27.5  016093  +0278   
4921  2022-06-01T02:53:00  29.98438  -95.36072       27.5  016093  +0272   
4922  2022-06-01T03:53:00  29.98438  -95.36072       27.5  016093  +0267   
4923  2022-06-01T04:53:00  29.98438  -95.36072       27.5  016093  +0272   
4924  2022-06-01T05:20:00  29.98438  -95.36072       27.5  016093  +0267   
4925  2022-06-01T05:53:00  29.98438  -95.36072       27.5  016093  +0267   
4926  2022-06-01T05:59:00  29.98438  -95.36072       27.5  999999  +9999   
4927  2022-06-01T05:59:00  29.98438  -95.36072       27.5  999999  +9999   

        DEW    SLP  DIR   SPD HEIGHT_AGL PRECIP_HOURS PRECIP  
4918  +0222  10117  140  0088      22000            0      0  
4919  +0228  10121  130  0082      22